In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler


import warnings
warnings.filterwarnings("ignore")

# Functions

In [2]:
#Standings
def cleanStandings(standings, year):
    standings = standings.dropna()
    standings["Rk"] = standings["Rk"].astype(int)
    standings = standings.set_index("Rk",drop=True)
    standings = standings[['Tm', 'W']]
    abbreviations = pd.read_csv("TeamAbbreviations.csv")
    standings = pd.merge(standings, abbreviations, on ='Tm').drop(columns=['Tm']).rename(columns={'Team': 'Tm'})
    standings['Tm'] = standings['Tm'] + f'{year}'
#     standings = standings.set_index('Tm')
    return standings

# Hitters
def cleanHitters(hittingPlayers):
    # Keep only a few columns
    hittingPlayers = hittingPlayers[['Name', 'Tm', 'PA', 'H', 'HR', 'BB', 'SB', 'BA', 'OBP', 'SLG']]
    hittingPlayers['Name'] = hittingPlayers['Name'].astype(str)

    hittingPlayers = hittingPlayers.dropna()
    # Clean player names
    hittingPlayers['Name'] = hittingPlayers['Name'].str.replace('*', '', regex=False)
    hittingPlayers['Name'] = hittingPlayers['Name'].str.replace('#', '', regex=False)
    #Set min PAs
    minPAs = 200
    hittingPlayers = hittingPlayers[hittingPlayers["PA"] > minPAs]
    hittingPlayers = hittingPlayers.drop(columns=['PA'])
    hittingPlayers = hittingPlayers[hittingPlayers['Tm'] != 'TOT']
    hittingPlayers = hittingPlayers.set_index('Name')
    return hittingPlayers

# Pitchers
def cleanPitchers(pitchingPlayers):
    # Keep only a few columns
    pitchingPlayers = pitchingPlayers[['Name', 'Tm', 'IP', 'H', 'HR', 'BB', 'SO', 'H9',  'HR9', 'BB9', 'SO9']]
    pitchingPlayers['Name'] = pitchingPlayers['Name'].astype(str)

    pitchingPlayers = pitchingPlayers.dropna()
    # Clean player names
    pitchingPlayers['Name'] = pitchingPlayers['Name'].str.replace('*', '', regex=False)
    pitchingPlayers['Name'] = pitchingPlayers['Name'].str.replace('#', '', regex=False)
    #Set min IPs
    minIPs = 73
    pitchingPlayers = pitchingPlayers[pitchingPlayers['IP'] > minIPs]
    pitchingPlayers = pitchingPlayers.drop(columns=['IP'])
    pitchingPlayers = pitchingPlayers[pitchingPlayers['Tm'] != 'TOT']
    pitchingPlayers = pitchingPlayers.set_index('Name')
    return pitchingPlayers

In [3]:
# Generate weights for each stat
def genWeights(count):
    while True:
        weights = []
        for i in range(count-1):
            weight = np.random.randint(1, 30)
            weights.append(weight)
        weights.append(100-sum(weights))
        random.shuffle(weights)
        if all(weight > 0 for weight in weights):
            return weights

In [4]:
# Create Value Metric
def playerValue(players, weights, typePlayer):
    hitting_stats = ['H', 'HR', 'BB', 'SB', 'BA', 'OBP', 'SLG']
    pitching_stats = ['H', 'HR', 'BB', 'SO', 'H9',  'HR9', 'BB9', 'SO9']
    scaler = MinMaxScaler()
    
    if typePlayer == "Hitter":
        # Normalize values due to discrepancy in ranges of values for different statistics
        for stat in players.columns[1:8]:
            players[[stat+ '_norm']] = scaler.fit_transform(players[[stat]])

        # Calculate and normalize weighted value
        for i, column in enumerate(players.columns[8:15]):
            mean_value = players[column].mean()
            weight = weights[i]
            players[column + '_val'] = (players[column] - mean_value) * weight
        
        players['Value'] = players.iloc[:, [15,16,17,18,19,20,21]].sum(axis=1)
        players = players.iloc[:, [0,1,2,3,4,5,6,7,22]]
        players = players.sort_values('Value', ascending=False)
    
    elif typePlayer == "Pitcher":
        # Negate Strikeouts because higher is better
        players['SOadj'] = -players['SO']
        players['SO9adj'] = -players['SO9']
        # Normalize values due to discrepancy in ranges of values for different statistics
        for stat in players.iloc[:, [1, 2, 3, 5, 6, 7, 9, 10]]:
            players[[stat+ '_norm']] = scaler.fit_transform(players[[stat]])
            
        for i, column in enumerate(players.columns[11:19]):
            mean_value = players[column].mean()
            weight = weights[i]
            players[column + '_val'] = (mean_value - players[column]) * weight
        
        
        players['Value'] = players.iloc[:, [19,20,21,22,23,24,25,26]].sum(axis=1)
        players = players.iloc[:, [0,1,2,3,4,5,6,7,8,27]]
        players = players.sort_values('Value', ascending=False)
    
    else:
        raise ValueError("Player type unavailable.")

    return players

In [5]:
def addPlayerValues(standings, values, typePlayer):
    # Merge dataframes on 'team'
    merged_df = pd.merge(standings, values, on='Tm')
    
    # Sort merged dataframe by team and value
    merged_df.sort_values(by=['Tm', 'Value'], ascending=[True, False], inplace=True)
    
    # Group by team and sum the top 5 values for each team
    sum_top_5 = merged_df.groupby('Tm').head(5).groupby('Tm')['Value'].sum().reset_index()

    # Add a new column to standings containing the summed values
    if typePlayer == "Hitter":
        standings['Top5HitterValue'] = standings['Tm'].map(sum_top_5.set_index('Tm')['Value'])
    
    elif typePlayer == "Pitcher":
        standings['Top5PitcherValue'] = standings['Tm'].map(sum_top_5.set_index('Tm')['Value'])
    
    else:
        raise ValueError("Player type unavailable.")

In [6]:
def calculate_r_squared(standings):
    x = standings['TotalValue'] 
    y = standings['W']
    m, b = np.polyfit(x, y, 1)
    y_pred = m*x + b
    r_squared = r2_score(y, y_pred)
    return r_squared

# SetUp

In [7]:
# Read in data
years = ['23', '22', '21', '19', '18', '17', '16', '15', '14', '13']
for year in years:
    standings = f'standings{year}'
    hittingPlayers = f'hittingPlayers{year}'
    pitchingPlayers = f'pitchingPlayers{year}'
    
    globals()[standings] = pd.read_csv(f'Baseball_Team_Standings_20{year}.csv')
    globals()[hittingPlayers] = pd.read_csv(f'Baseball_Hitting_Player_20{year}.csv')
    globals()[pitchingPlayers] = pd.read_csv(f'Baseball_Pitching_Player_20{year}.csv')

In [8]:
# Clean all hitters, pitchers and standings
years = ['23', '22', '21', '19', '18', '17', '16', '15', '14', '13']
for year in years:
    standings = f'standings{year}'
    hittingPlayers = f'hittingPlayers{year}'
    pitchingPlayers = f'pitchingPlayers{year}'
    
    globals()[standings] = cleanStandings(globals()[standings], year)
    
    globals()[hittingPlayers] = cleanHitters(globals()[hittingPlayers])
    globals()[hittingPlayers]['Tm'] = globals()[hittingPlayers]['Tm'] + f'{year}'
    
    globals()[pitchingPlayers] = cleanPitchers(globals()[pitchingPlayers])
    globals()[pitchingPlayers]['Tm'] = globals()[pitchingPlayers]['Tm'] + f'{year}'

# Loop

In [14]:
years = []
hitter_weights_list = []
pitcher_weights_list = []
r_squared_values = []
results = pd.DataFrame()

for runs in range(1, 1000):

    # Generate hitter and pitcher weights
    hitter_weights = genWeights(7)
    pitcher_weights = genWeights(8)

    # Iterate over years
    for year in range(13, 24):
        if year != 20:
            hitters_data = globals().get(f'hittingPlayers{year}')
            pitchers_data = globals().get(f'pitchingPlayers{year}')
            standings_data = globals().get(f'standings{year}')
        
            hitterValues = playerValue(hitters_data, hitter_weights, "Hitter")
            pitcherValues = playerValue(pitchers_data, pitcher_weights, "Pitcher")

            # Add player values to standings DataFrame
            addPlayerValues(standings_data, hitterValues, "Hitter")
            addPlayerValues(standings_data, pitcherValues, "Pitcher")
    
            # Calculate total value and R-squared value
            standings_data['TotalValue'] = standings_data['Top5HitterValue'] + standings_data['Top5PitcherValue']
            r_squared = calculate_r_squared(standings_data)
    
            # Append data to lists
            years.append(year)
            hitter_weights_list.append(hitter_weights)
            pitcher_weights_list.append(pitcher_weights)
            r_squared_values.append(r_squared)

            # Create DataFrame
            temp = pd.DataFrame({
                'Year': years,
                'Run': runs,
                'Hitter Weights': hitter_weights_list,
                'Pitcher Weights': pitcher_weights_list,
                'R-squared': r_squared_values
            })
            results = pd.concat([results, temp], ignore_index=True)

            # Reset lists for next iteration
            years = []
            hitter_weights_list = []
            pitcher_weights_list = []
            r_squared_values = []

# Sort results
results = results.sort_values('R-squared', ascending=False)

# Display DataFrame
print(results.head(20))

      Year  Run               Hitter Weights                 Pitcher Weights  \
7228    22  723    [25, 6, 21, 8, 1, 27, 12]    [8, 12, 4, 6, 12, 25, 27, 6]   
988     22   99    [10, 12, 28, 1, 4, 40, 5]    [17, 5, 2, 3, 23, 27, 22, 1]   
838     22   84   [28, 10, 19, 8, 13, 16, 6]     [4, 21, 4, 8, 2, 16, 42, 3]   
2148    22  215   [22, 4, 11, 18, 29, 4, 12]    [6, 6, 10, 19, 6, 21, 28, 4]   
1708    22  171   [15, 16, 25, 1, 13, 29, 1]   [1, 8, 11, 31, 19, 10, 17, 3]   
1858    22  186    [29, 9, 15, 13, 5, 22, 7]  [1, 2, 14, 19, 11, 16, 22, 15]   
7268    22  727   [21, 22, 14, 12, 18, 8, 5]   [6, 4, 14, 15, 11, 28, 19, 3]   
4388    22  439   [13, 21, 24, 8, 18, 6, 10]    [4, 5, 18, 1, 21, 21, 26, 4]   
7408    22  741  [11, 20, 17, 14, 22, 11, 5]   [1, 5, 13, 27, 13, 18, 16, 7]   
8788    22  879   [17, 22, 19, 8, 22, 11, 1]   [8, 3, 14, 19, 23, 15, 17, 1]   
5848    22  585   [23, 1, 3, 20, 17, 19, 17]    [13, 1, 15, 7, 3, 14, 46, 1]   
458     22   46    [5, 7, 21, 2, 17, 23,

hitting_stats = ['H', 'HR', 'BB', 'SB', 'BA', 'OBP', 'SLG']

pitching_stats = ['H', 'HR', 'BB', 'SO', 'H9',  'HR9', 'BB9', 'SO9']